# Lesson 1: Batching Commands with Pipelines

# Introduction to Batching Commands with Pipelines

Welcome! In this lesson, we are going to delve into a feature of Redis that can significantly boost your application's performance — **pipelines**. Pipelines allow you to send multiple commands to the Redis server without waiting for a response after each command. Instead, you collect a batch of commands and send them all at once, then read all the replies in a single step. This approach can make your application more efficient and responsive.

Ready to optimize your Redis usage? Let's get started!

## What You'll Learn
In this lesson, we will explore how to use Redis pipelines to batch commands. Specifically, you will learn how to:

- 🛠️ Initialize a Redis connection and pipeline.
- 📊 Batch multiple commands together.
- 🚀 Execute the batched commands efficiently.

Here's a quick example to give you an overview. Consider a scenario where you need to update the number of courses completed and set a user's name. Normally, you would execute these commands one by one. With pipelines, you can batch them like this:

```python
import redis

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

# Initialize values
client.set('user', '')
client.set('courses_completed', 1)

# Use the pipeline without a context manager
pipe = client.pipeline()
pipe.incr('courses_completed')
pipe.set('user', 'John')
try:
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction error: {e}")
finally:
    pipe.close()

# Retrieve and print updated values
courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')
```

This sample code demonstrates how to connect to Redis, batch commands in a pipeline, and then execute them all together for better performance.

### Explanation:
- First, we create a pipeline and add the commands to increment the number of courses completed and set the user's name.
- Then, we execute the pipeline and print the results.
- Finally, we retrieve the updated values and display them.

Notice that we use the `pipe.execute()` method to execute the batched commands. This method sends all the commands to the Redis server and retrieves the results in a single step.

After that, we use the `pipe.close()` method to close the pipeline and release the resources. This is important to avoid memory leaks and ensure proper cleanup. Another useful method is `pipe.reset()`, which resets the pipeline and the intermediate state, but you can still reuse it.

### Transaction Results:
Let's understand the successful transaction result, which will be:

```
Transaction results: [2, True]
```

- The first element is the result of the `incr` command, which increments the value by 1, hence the value `2`.
- The second element is the result of the `set` command, which returns `True` to indicate success.

## Why It Matters

Efficiency is key in any application, and being able to execute multiple Redis commands in one go can save you a lot of time and resources. This is particularly important in real-time applications where latency can be a critical factor. By mastering pipelines, you can enhance the responsiveness of your applications and provide a smoother user experience.

Exciting, right? Ready to see how much you can optimize your Redis interactions? Let's move on to the practice section and put these concepts into action!


## Running a Redis Transaction

Great job understanding the basics of Redis transactions! Let's run the code you saw in the lesson to see it in action.

Redis transactions allow you to execute a batch of commands as a single unit. In the code, we set some initial values, add a new user, and increment the number of completed courses within a single transaction.

The commands in the pipeline are executed together without waiting for the response to each command.

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

client.set('user', '')
client.set('courses_completed', 1)

pipe = client.pipeline()
pipe.set('user', 'John')
pipe.incr('courses_completed')

try:
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction results: {e}")
finally:
    pipe.close()

courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')

```

## Adding a Decrement in Pipeline

Nice progress! You’ve got the hang of executing Redis transactions within a pipeline. Now, let’s add a twist.

Rather than just setting a value and incrementing it, let's include a decrement operation in our pipeline. Modify the given code to decrement the missions_ongoing key by 2 in the pipeline.

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

client.set('user', '')
client.set('courses_completed', 1)
client.set('missions_ongoing', 5)

pipe = client.pipeline()
pipe.incr('courses_completed')
pipe.set('user', 'John')
# TODO: Add a command to decrement missions_ongoing by 2 in the pipeline

try:
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction results: {e}")
finally:
    pipe.close()

courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')
missions_ongoing = client.get('missions_ongoing').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')
print(f'Missions ongoing: {missions_ongoing}')

```

Here's the modified version of your code that includes a decrement operation for the `missions_ongoing` key in the pipeline. The key is decremented by 2 as requested:

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

# Initialize values
client.set('user', '')
client.set('courses_completed', 1)
client.set('missions_ongoing', 5)

# Create pipeline
pipe = client.pipeline()

# Add commands to the pipeline
pipe.incr('courses_completed')       # Increment courses_completed by 1
pipe.set('user', 'John')             # Set the user's name
pipe.decrby('missions_ongoing', 2)   # Decrement missions_ongoing by 2

try:
    # Execute all the batched commands
    results = pipe.execute()
    print(f"Transaction results: {results}")
except Exception as e:
    print(f"Transaction error: {e}")
finally:
    pipe.close()

# Retrieve and print updated values
courses_completed = client.get('courses_completed').decode('utf-8')
user = client.get('user').decode('utf-8')
missions_ongoing = client.get('missions_ongoing').decode('utf-8')

print(f'Courses completed: {courses_completed}')
print(f'User: {user}')
print(f'Missions ongoing: {missions_ongoing}')
```

### Key Changes:
- Added `pipe.decrby('missions_ongoing', 2)` to decrement `missions_ongoing` by 2 within the pipeline.

### Expected Transaction Results:
```
Transaction results: [2, True, 3]
```
- The first result (`2`) is from incrementing `courses_completed`.
- The second result (`True`) is from setting the user to "John".
- The third result (`3`) is from decrementing `missions_ongoing` (5 - 2 = 3).

### Final Output:
- `Courses completed: 2`
- `User: John`
- `Missions ongoing: 3`

This approach shows how to mix increment, set, and decrement operations in a single Redis pipeline!

## Debug Redis Pipeline Errors

## Complete the Redis Pipeline

## Write a Redis Pipeline Script

## Improve Efficiency with Redis Pipelines